In [56]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

In [79]:
from lifelines import CoxPHFitter 
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test, multivariate_logrank_test

In [26]:
attention_score_path = "/data/projects/GDC_Cancer_Wise/Brain/Data/Visualization_data/Acceptor/Prediction_result/atten_score80.npy"
prediction_path = "/data/projects/GDC_Cancer_Wise/Brain/Data/Visualization_data/Acceptor/Prediction_result/pred_results.npy"
data_path = "/data/projects/GDC_Cancer_Wise/Brain/Data/Visualization_data/Acceptor/dev.tsv"
raw_data_path = "/data/projects/GDC_Cancer_Wise/Brain/Data/Visualization_data/Acceptor/raw_data.tsv"
data_statistics = "/data/projects/GDC_Cancer_Wise/Brain/Data/Postprocessing_result/Acceptor/important_snp.tsv"

In [38]:
attention_data = np.load(attention_score_path)
pred_result = np.load(prediction_path)
data = pd.read_csv(data_path, sep="\t") 
raw_data = pd.read_csv(raw_data_path, sep="\t")
df_snp = pd.read_csv(data_statistics, sep="\t")

In [28]:
attention_data

array([[0.01490009, 0.00858043, 0.00673186, ..., 0.00683163, 0.00867788,
        0.01443886],
       [0.0142613 , 0.00871497, 0.00696233, ..., 0.02421607, 0.03467445,
        0.0659325 ],
       [0.02704413, 0.01780735, 0.01480795, ..., 0.08836011, 0.13167963,
        0.26301744],
       ...,
       [0.01499418, 0.01007978, 0.00943549, ..., 0.00348013, 0.00170559,
        0.00089852],
       [0.01763524, 0.02300015, 0.01869645, ..., 0.03688714, 0.04093168,
        0.06625   ],
       [0.01098231, 0.01113181, 0.00911861, ..., 0.01283992, 0.0114428 ,
        0.01403958]])

In [29]:
attention_data = attention_data[:, 1:]

In [30]:
len(attention_data[0])

79

In [31]:
pred_result

array([7.0022678e-01, 3.2903254e-02, 9.8029608e-01, ..., 9.6866232e-04,
       7.9034662e-01, 3.8581815e-02], dtype=float32)

In [32]:
#seq_wise_vis_dir = "/data/projects/Enhancer/DNABERT_Output/Visualization_Enhancer_Encode/Predictions/V"
seqwise_attention_path = os.path.dirname(attention_score_path)+"/Nucleotide_wise_attention_map"
if not os.path.exists(seqwise_attention_path):
    os.makedirs(seqwise_attention_path)

In [33]:
len(attention_data)

1652

In [34]:
raw_data

,Unnamed: 0,chr,strand,Transcript_ID,Acceptor_start,Acceptor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq,Ref_Score,Alt_Score,patient_id
0,0,chr1,+,"ENST00000246151.9, ENST00000374524.1, ENST0000...",23786274,23786353,23786322,23786323,G,A,ATAATTCATACCTTCTTTCCCTATTCCTGTCATCCTCTAGAATTTC...,ATAATTCATACCTTCTTTCCCTATTCCTGTCATCCTCTAGAATTTC...,0.700227,0.032903,"['6fee80f8-11cb-4de7-aded-e586708d2368', 'f98d..."
1,1,chr1,+,"ENST00000261464.10, ENST00000336184.6, ENST000...",211371261,211371340,211371265,211371266,A,G,ACTAATTTTTTAAACATGATTATCCATTTTGTAATGAAAGGTTTTT...,ACTAGTTTTTTAAACATGATTATCCATTTTGTAATGAAAGGTTTTT...,0.980296,0.004916,"['e849abef-b55e-4124-b760-09101a7c4390', 'b020..."
2,2,chr1,+,ENST00000271588.9,186078066,186078145,186078129,186078130,C,T,TAAGATTAGAGGAGTAAACATAGTGGGATATTATTTTCAGGAGGGC...,TAAGATTAGAGGAGTAAACATAGTGGGATATTATTTTCAGGAGGGC...,0.992662,0.020286,"['0332b017-17d5-4083-8fc4-9d6f8fdbbbde', '8faa..."
3,3,chr1,+,"ENST00000292357.8, ENST00000338302.7, ENST0000...",156912729,156912808,156912797,156912798,G,A,GCCAAGATGCCATTCTGAGTGAGCACCCCATTCCACACAGGAATCC...,GCCAAGATGCCATTCTGAGTGAGCACCCCATTCCACACAGGAATCC...,0.974504,0.078847,"['5fd77ba9-5015-4d8b-86a0-582e5c76bdd6', '54c6..."
4,4,chr1,+,ENST00000294428.7,64804693,64804772,64804703,64804707,TAAA,TAA,TTTTCAAAATTAAACTGACAACGTTTTGTCATTTTCACAGAGCTCA...,TTTTCAAAATTAACTGACAACGTTTTGTCATTTTCACAGAGCTCAG...,0.999974,0.064063,"['d85192d7-683f-4214-b8de-ae2e4ff2405c', '25be..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,821,chrX,-,ENST00000438499.1,6668090,6668169,6668146,6668147,G,T,GATTCTCCTCCAATGCATCACATTTTTATTCCCCTGTCCCTGTGGG...,GATTCTCCTCCAATGCATCACATTTTTATTCCCCTGTCCCTGTGGG...,0.741388,0.010471,"['e7b36f2d-9ab1-4a46-b669-65367b841752', 'c087..."
822,822,chrX,-,ENST00000480178.1,9748034,9748113,9748044,9748045,T,G,AGGAAATGCTTGAGACAGTTCCAGCGGCCACATCTCCCTCTGTGTG...,AGGAAATGCTGGAGACAGTTCCAGCGGCCACATCTCCCTCTGTGTG...,0.636375,0.043710,"['cf245938-dd0c-42fe-9a39-09f29ff12f23', '0bce..."
823,823,chrX,-,ENST00000494525.1,21733718,21733797,21733771,21733772,G,A,TCCTCTTCAATTCTTTGATATCACTGAGTTCTTTTACCACTGGAAA...,TCCTCTTCAATTCTTTGATATCACTGAGTTCTTTTACCACTGGAAA...,0.702985,0.007422,['41d50a3c-49fc-4d8a-9c25-bbbcd3a5da6f']
824,824,chrX,-,ENST00000624824.1,15702688,15702767,15702716,15702717,A,G,TGGCACACGGCGCCTTGTGCACTTTCCAACATTTTGATGCTAACCG...,TGGCACACGGCGCCTTGTGCACTTTCCAGCATTTTGATGCTAACCG...,0.939159,0.000969,"['0332b017-17d5-4083-8fc4-9d6f8fdbbbde', '8faa..."


In [35]:
merged_list = list(zip(raw_data['reference_seq'], raw_data['alt_seq']))
all_seq = [item.upper() for tup in merged_list for item in tup ]
kmers = [list(seq) for seq in all_seq]

num_columns = attention_data.shape[1]
print(num_columns)

79


In [ ]:
# Define the length of the region to be visualized around the variant
buffer_length = 10

# Iterate over sequences and attention scores
for i, sequence in enumerate(kmers):
    # Calculate the relative positions of variant start and end within the sequence
    relative_variant_start = raw_data['varinat_start'].iloc[i // 2] - raw_data['Acceptor_start'].iloc[i // 2]
    relative_variant_end = raw_data['variant_end'].iloc[i // 2] - raw_data['Acceptor_start'].iloc[i // 2]
    #print(relative_variant_start, relative_variant_end)

    # Adjust indices for buffer length, ensuring they are within sequence boundaries
    start_index = max(0, relative_variant_start - buffer_length)
    end_index = min(len(sequence), relative_variant_end + buffer_length)

    # Check if slicing range is valid
    if start_index >= end_index:
        print("Invalid slice range, skipping...")
        continue

    # Slice the sequence and attention data
    sliced_sequence = sequence[start_index:end_index]
    sliced_attention = attention_data[i, start_index:end_index]
    #print(sliced_sequence)
    #print(sliced_attention)

    # Plot heatmap
    plt.figure(figsize=(18, 3))
    sns.heatmap([sliced_attention], cmap='YlGnBu', cbar=True, annot=[sliced_sequence], fmt='', annot_kws={"size": 8})
    original_start_index=  max(raw_data['Acceptor_start'].iloc[i//2], raw_data['varinat_start'].iloc[i//2] - buffer_length)
    original_end_index=  min(raw_data['Acceptor_end'].iloc[i//2], raw_data['variant_end'].iloc[i//2] + buffer_length)
    seq_type = "Reference Sequence" if i % 2 == 0 else "Alternative Sequence"
    title = f"{seq_type} around Variant {raw_data['chr'].iloc[i // 2]}:{original_start_index}-{original_end_index}"
    #print(title)
    plt.title(title)
    plt.savefig(seqwise_attention_path + "/" + f"{title}.png", bbox_inches='tight')
    plt.close()
    #input()

In [59]:
#seq_wise_vis_dir = "/data/projects/Enhancer/DNABERT_Output/Visualization_Enhancer_Encode/Predictions/V"
seqwise_attention_path = os.path.dirname(attention_score_path)+"/Nucleotide_wise_attention_map_paried_sequence"
if not os.path.exists(seqwise_attention_path):
    os.makedirs(seqwise_attention_path)

In [60]:
df_snp

,Column,Max Value,Non-NaN Count,Non-NaN Indices,Percentage,Variant Region Difference
0,chr16_58267438_58267517_58267455_58267458,36.161531,32,"[6, 7, 25, 32, 49, 63, 65, 80, 97, 115, 128, 1...",10.958904,3
1,chr10_73797336_73797415_73797341_73797342,35.699415,3,"[17, 219, 266]",1.027397,1
2,chr12_23190232_23190311_23190241_23190245,34.806988,4,"[26, 163, 196, 290]",1.369863,4
3,chr4_52750790_52750869_52750817_52750818,34.397403,2,"[76, 240]",0.684932,1
4,chr3_175460004_175460083_175460009_175460012,34.386584,2,"[122, 257]",0.684932,3
...,...,...,...,...,...,...
3780,chr1_209623686_209623765_209623748_209623749,5.003059,6,"[73, 108, 140, 156, 167, 206]",2.054795,1
3781,chr6_29828502_29828581_29828528_29828529,5.002682,193,"[0, 1, 2, 3, 6, 7, 10, 11, 13, 14, 15, 16, 17,...",66.095890,1
3782,chr19_35259199_35259278_35259204_35259205,5.002419,1,[44],0.342466,1
3783,chr13_110492029_110492108_110492064_110492065,5.000697,22,"[43, 52, 53, 64, 66, 72, 98, 106, 115, 118, 13...",7.534247,1


In [83]:
pd.set_option('display.max_columns', None)
df_clinical = pd.read_csv("/data/projects/GDC_Cancer_Wise/Brain/Clinical_files/TCGA_XENA_LGG_survival.txt", sep="\t")
#df_clinical = df_clinical[~(df_clinical.iloc[:, 3:] == "'--").all(axis=1)]
df_clinical

,sample,_PATIENT,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
0,TCGA-CS-4938-01,TCGA-CS-4938,0,3574.0,0.0,3574.0,NaN,NaN,0,3574.0,NaN
1,TCGA-CS-4941-01,TCGA-CS-4941,1,234.0,1.0,234.0,NaN,NaN,1,9.0,NaN
2,TCGA-CS-4942-01,TCGA-CS-4942,1,1335.0,1.0,1335.0,NaN,NaN,1,1184.0,NaN
3,TCGA-CS-4943-01,TCGA-CS-4943,1,1106.0,1.0,1106.0,NaN,NaN,1,1106.0,NaN
4,TCGA-CS-4944-01,TCGA-CS-4944,0,1828.0,0.0,1828.0,NaN,NaN,0,1828.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
524,TCGA-WY-A85A-01,TCGA-WY-A85A,0,1320.0,0.0,1320.0,NaN,NaN,0,1320.0,NaN
525,TCGA-WY-A85B-01,TCGA-WY-A85B,0,1393.0,0.0,1393.0,0.0,1393.0,0,1393.0,NaN
526,TCGA-WY-A85C-01,TCGA-WY-A85C,0,1426.0,0.0,1426.0,NaN,NaN,1,809.0,NaN
527,TCGA-WY-A85D-01,TCGA-WY-A85D,0,1147.0,0.0,1147.0,NaN,NaN,1,887.0,NaN


In [82]:
list(df_clinical.columns)

['case_id',
 'case_submitter_id',
 'project_id',
 'age_at_index',
 'age_is_obfuscated',
 'cause_of_death',
 'cause_of_death_source',
 'country_of_residence_at_enrollment',
 'days_to_birth',
 'days_to_death',
 'ethnicity',
 'gender',
 'occupation_duration_years',
 'premature_at_birth',
 'race',
 'vital_status',
 'weeks_gestation_at_birth',
 'year_of_birth',
 'year_of_death',
 'adrenal_hormone',
 'age_at_diagnosis',
 'ajcc_clinical_m',
 'ajcc_clinical_n',
 'ajcc_clinical_stage',
 'ajcc_clinical_t',
 'ajcc_pathologic_m',
 'ajcc_pathologic_n',
 'ajcc_pathologic_stage',
 'ajcc_pathologic_t',
 'ajcc_staging_system_edition',
 'anaplasia_present',
 'anaplasia_present_type',
 'ann_arbor_b_symptoms',
 'ann_arbor_b_symptoms_described',
 'ann_arbor_clinical_stage',
 'ann_arbor_extranodal_involvement',
 'ann_arbor_pathologic_stage',
 'best_overall_response',
 'breslow_thickness',
 'burkitt_lymphoma_clinical_variant',
 'child_pugh_classification',
 'circumferential_resection_margin',
 'classificatio

In [76]:
def KMPlot(df_kmf, saved_path):
    kmf = KaplanMeierFitter()
    T = df_kmf["time"]
    E = df_kmf["status"]
    plt.figure(figsize=(20,20))
    for i in range(2):
        T = df_kmf[df_kmf['label']==i]['time'].to_list()
        E = df_kmf[df_kmf['label']==i]['status'].to_list()
        if(i==1):
            kmf.fit(T, E, label="Affected Patients")
        else if (i==0):
            kmf.fit(T, E, label="Non-affected Patients")
        ax = kmf.plot_survival_function(ax=ax)
    result = multivariate_logrank_test(df_kmf['time'], df_kmf['label'], df_kmf['status'])
    #print("##",result.test_statistic)
    print(k,result.p_value)
    #print("IO",result.print_summary())
    plt.title("KM Plot for cluster number ="+str(k)+"\n[pvalue: {}]".format(result.p_value));

,Unnamed: 0,chr,strand,Transcript_ID,Acceptor_start,Acceptor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq,Ref_Score,Alt_Score,patient_id
0,0,chr1,+,"ENST00000246151.9, ENST00000374524.1, ENST0000...",23786274,23786353,23786322,23786323,G,A,ATAATTCATACCTTCTTTCCCTATTCCTGTCATCCTCTAGAATTTC...,ATAATTCATACCTTCTTTCCCTATTCCTGTCATCCTCTAGAATTTC...,0.700227,0.032903,"['6fee80f8-11cb-4de7-aded-e586708d2368', 'f98d..."
1,1,chr1,+,"ENST00000261464.10, ENST00000336184.6, ENST000...",211371261,211371340,211371265,211371266,A,G,ACTAATTTTTTAAACATGATTATCCATTTTGTAATGAAAGGTTTTT...,ACTAGTTTTTTAAACATGATTATCCATTTTGTAATGAAAGGTTTTT...,0.980296,0.004916,"['e849abef-b55e-4124-b760-09101a7c4390', 'b020..."
2,2,chr1,+,ENST00000271588.9,186078066,186078145,186078129,186078130,C,T,TAAGATTAGAGGAGTAAACATAGTGGGATATTATTTTCAGGAGGGC...,TAAGATTAGAGGAGTAAACATAGTGGGATATTATTTTCAGGAGGGC...,0.992662,0.020286,"['0332b017-17d5-4083-8fc4-9d6f8fdbbbde', '8faa..."
3,3,chr1,+,"ENST00000292357.8, ENST00000338302.7, ENST0000...",156912729,156912808,156912797,156912798,G,A,GCCAAGATGCCATTCTGAGTGAGCACCCCATTCCACACAGGAATCC...,GCCAAGATGCCATTCTGAGTGAGCACCCCATTCCACACAGGAATCC...,0.974504,0.078847,"['5fd77ba9-5015-4d8b-86a0-582e5c76bdd6', '54c6..."
4,4,chr1,+,ENST00000294428.7,64804693,64804772,64804703,64804707,TAAA,TAA,TTTTCAAAATTAAACTGACAACGTTTTGTCATTTTCACAGAGCTCA...,TTTTCAAAATTAACTGACAACGTTTTGTCATTTTCACAGAGCTCAG...,0.999974,0.064063,"['d85192d7-683f-4214-b8de-ae2e4ff2405c', '25be..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,821,chrX,-,ENST00000438499.1,6668090,6668169,6668146,6668147,G,T,GATTCTCCTCCAATGCATCACATTTTTATTCCCCTGTCCCTGTGGG...,GATTCTCCTCCAATGCATCACATTTTTATTCCCCTGTCCCTGTGGG...,0.741388,0.010471,"['e7b36f2d-9ab1-4a46-b669-65367b841752', 'c087..."
822,822,chrX,-,ENST00000480178.1,9748034,9748113,9748044,9748045,T,G,AGGAAATGCTTGAGACAGTTCCAGCGGCCACATCTCCCTCTGTGTG...,AGGAAATGCTGGAGACAGTTCCAGCGGCCACATCTCCCTCTGTGTG...,0.636375,0.043710,"['cf245938-dd0c-42fe-9a39-09f29ff12f23', '0bce..."
823,823,chrX,-,ENST00000494525.1,21733718,21733797,21733771,21733772,G,A,TCCTCTTCAATTCTTTGATATCACTGAGTTCTTTTACCACTGGAAA...,TCCTCTTCAATTCTTTGATATCACTGAGTTCTTTTACCACTGGAAA...,0.702985,0.007422,['41d50a3c-49fc-4d8a-9c25-bbbcd3a5da6f']
824,824,chrX,-,ENST00000624824.1,15702688,15702767,15702716,15702717,A,G,TGGCACACGGCGCCTTGTGCACTTTCCAACATTTTGATGCTAACCG...,TGGCACACGGCGCCTTGTGCACTTTCCAGCATTTTGATGCTAACCG...,0.939159,0.000969,"['0332b017-17d5-4083-8fc4-9d6f8fdbbbde', '8faa..."


In [75]:
buffer_length = 20  # Define buffer length

# Iterate over pairs of sequences (reference and alternative) and attention scores
for i in range(0, len(kmers), 2):
    if(i%20==0):
        print(i)
    sequence_ref = kmers[i]
    sequence_alt = kmers[i + 1]
    index = raw_data['chr'].iloc[i // 2]+"_"+str(raw_data['Acceptor_start'].iloc[i // 2])+"_"+str(raw_data['Acceptor_end'].iloc[i // 2])+"_"+str(raw_data['varinat_start'].iloc[i // 2])+"_"+str(raw_data['variant_end'].iloc[i // 2])
    #print(index)
    percentage_value = df_snp[df_snp['Column']==index]['Percentage'].values[0]
    #print(percentage_value)
    lower_bound = math.floor(percentage_value)
    upper_bound = math.ceil(percentage_value)
    #print(lower_bound , upper_bound)
    directory_name = f"{lower_bound}_{upper_bound}"
    #print("##", directory_name)

    # Calculate relative positions of variant start and end within the sequence
    relative_variant_start = raw_data['varinat_start'].iloc[i // 2] - raw_data['Acceptor_start'].iloc[i // 2]
    relative_variant_end = raw_data['variant_end'].iloc[i // 2] - raw_data['Acceptor_start'].iloc[i // 2]

    # Adjust indices for buffer length, ensuring they are within sequence boundaries
    start_index = max(0, relative_variant_start - buffer_length)
    end_index = min(len(sequence_ref), relative_variant_end + buffer_length)

    # Check if slicing range is valid
    if start_index >= end_index:
        print("Invalid slice range, skipping...")
        continue

    # Slice the sequences and ` data
    sliced_sequence_ref = sequence_ref[start_index:end_index]
    sliced_sequence_alt = sequence_alt[start_index:end_index]
    sliced_attention_ref = attention_data[i, start_index:end_index]
    sliced_attention_alt = attention_data[i + 1, start_index:end_index]
    
    original_start_index=  max(raw_data['Acceptor_start'].iloc[i//2], raw_data['varinat_start'].iloc[i//2] - buffer_length)
    original_end_index=  min(raw_data['Acceptor_end'].iloc[i//2], raw_data['variant_end'].iloc[i//2] + buffer_length)
    
    sub_seqwise_attention_path = seqwise_attention_path + "/" +directory_name
    if not os.path.exists(sub_seqwise_attention_path):
        os.makedirs(sub_seqwise_attention_path)

    
    # Determine the number of ticks needed, with a tick for every 5 nucleotides
    num_ticks = (original_end_index - original_start_index) // 5 + 1  # Ensure we include the end index

    # Generate the tick positions and labels
    tick_positions = [i * 5 for i in range(num_ticks)]  # Positions in the sequence
    tick_labels = [original_start_index + pos for pos in tick_positions]  # Map to actual nucleotide positions

    # Ensure tick positions and labels do not exceed the length of the sequence or the end index
    tick_positions = [pos for pos, label in zip(tick_positions, tick_labels) if label <= original_end_index]
    tick_labels = [label for label in tick_labels if label <= original_end_index]
    # print(plot_range)
    # print(tick_positions)
    # print(tick_labels)
    # Create a new figure for each iteration
    fig, axs = plt.subplots(2, 1, figsize=(18, 3))
    plt.subplots_adjust(hspace=1)  # Adjust spacing between subplots
    
    #y_position = 1.08
    
        # Add text for the Acceptor Region
    plt.text(
        x=0.5, 
        y=1.08, 
        s=f"Acceptor Region: {raw_data['Acceptor_start'].iloc[i // 2]}-{raw_data['Acceptor_end'].iloc[i // 2]}", 
        horizontalalignment='center', 
        verticalalignment='center', 
        transform=plt.gcf().transFigure
    )
    #y_position -= 0.04  # Adjust the y position for the next line

    # Add text for the Variant Region
    # Replace variant_region_info with the appropriate variable or value
    plt.text(
        x=0.5, 
        y=1.04, 
        s=f"Variant Region: {raw_data['varinat_start'].iloc[i // 2]}-{raw_data['variant_end'].iloc[i // 2]}", 
        horizontalalignment='center', 
        verticalalignment='center', 
        transform=plt.gcf().transFigure
    )
    #y_position -= 0.04  # Adjust the y position for the next line

    # Add text for the Region Plotted
    # Replace plotted_region_info with the appropriate variable or value
    plt.text(
        x=0.5, 
        y=1.00, 
        s=f"Region Plotted: {original_start_index}-{original_end_index}", 
        horizontalalignment='center', 
        verticalalignment='center', 
        transform=plt.gcf().transFigure
    )   # Create subplots
    #y_position -= 0.08  # Adjust the y position for the next line
    
    
    # Plot heatmap for reference sequence
    sns.heatmap([sliced_attention_ref], cmap='YlGnBu', cbar=True, annot=[sliced_sequence_ref], fmt='', annot_kws={"size": 8}, ax=axs[0])
    axs[0].set_xticks(tick_positions)  # Set the tick positions on the x-axis
    axs[0].set_xticklabels(tick_labels)  # Set the tick labels

    # Plot heatmap for alternative sequence
    sns.heatmap([sliced_attention_alt], cmap='YlGnBu', cbar=True, annot=[sliced_sequence_alt], fmt='', annot_kws={"size": 8}, ax=axs[1])
    axs[1].set_title(f"Alternative Sequence {raw_data['chr'].iloc[i // 2]}:{original_start_index}-{original_end_index}")
    axs[1].set_xticks(tick_positions)  # Set the tick positions on the x-axis for the second subplot
    axs[1].set_xticklabels(tick_labels)  # Set the tick labels


    # Save the plot
    plt.savefig(sub_seqwise_attention_path + "/" + f"Paired_sequence_{raw_data['chr'].iloc[i // 2]}_{original_start_index}_{original_end_index}.png", bbox_inches='tight')
    plt.close()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
